In [2]:
import pandas as pd
import numpy as np

def TE_dataframe_dask(df: pd.DataFrame,
                      feature_name: str,
                      target_name: str,
                      w_smoothing: int = 20,
                      dt=np.float64) -> pd.Series:

    counts_and_means = df[[target_name, feature_name]].groupby(feature_name)[target_name].agg(['count', 'mean'])
    counts_and_means["total_mean"] = df[target_name].mean()  # redundant but necessary
    TE_map = counts_and_means.apply(lambda cm: (cm["count"]*cm["mean"]+w_smoothing*cm["total_mean"])/(cm["count"]+w_smoothing),
                                    axis=1
                                    )
    # the necessary painful part - join on non index
    #TE_series = df[feature_name].apply(lambda feat: TE_map[feat])
    return TE_map, counts_and_means    # all lazy all dask, should be fine, evaluated in the end when all the things are merged

In [3]:
#df = pd.read_parquet('preprocessed/part.0.parquet')
df = pd.read_parquet('preprocessed_features/part.0.parquet')
df

,has_reply,bert_token_len,has_retweet,has_retweet_comment,has_like,n_photos,n_videos,n_gifs,reply_age,TE_language_has_reply
idx,,,,,,,,,,
1,False,24,False,False,True,NaN,NaN,NaN,0,0.028091
2,False,27,True,False,False,1.0,0.0,0.0,0,0.025616
3,False,59,False,False,False,NaN,NaN,NaN,0,0.032983
4,False,42,False,False,False,0.0,1.0,0.0,0,0.067939
5,False,25,False,False,True,1.0,0.0,0.0,0,0.067939
...,...,...,...,...,...,...,...,...,...,...
258285,False,44,False,False,False,NaN,NaN,NaN,0,0.028091
258286,False,44,False,False,False,NaN,NaN,NaN,0,0.028091
258287,False,47,False,False,False,NaN,NaN,NaN,0,0.025616


In [2]:
df["retweet"] = df["retweet"].apply(lambda a: a > 0.)
tmap, cms = TE_dataframe_dask(df, "language", "retweet")
tmap.name = "azaza"
tmap

NameError: name 'df' is not defined

In [22]:
df["language"].join(tmap, on='language', how='left')

AttributeError: 'Series' object has no attribute 'join'

In [5]:
df.groupby('reply_age').count()

,has_reply,bert_token_len,has_retweet,has_retweet_comment,has_like,n_photos,n_videos,n_gifs,TE_language_has_reply
reply_age,,,,,,,,,
0,251131,251131,251131,251131,251131,96500,96500,96500,251131
5,1,1,1,1,1,0,0,0,1
7,4,4,4,4,4,1,1,1,4
8,4,4,4,4,4,0,0,0,4
9,2,2,2,2,2,1,1,1,2
...,...,...,...,...,...,...,...,...,...
603124,1,1,1,1,1,0,0,0,1
627611,1,1,1,1,1,0,0,0,1
661431,1,1,1,1,1,1,1,1,1


In [8]:
df.groupby('medias').count()

,bert_base_multilingual_cased_tokens,hashtags,tweet_id,links,domains,type,language,timestamp,a_user_id,a_follower_count,...,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like
medias,,,,,,,,,,,,,,,,,,,,,
GIF,3281,945,3281,395,395,3281,3281,3281,3281,3281,...,3281,3281,3281,3281,3281,3281,3281,3281,3281,3281
GIF\tGIF,2,1,2,1,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Photo,51982,14101,51982,6864,6864,51982,51982,51982,51982,51982,...,51982,51982,51982,51982,51982,51982,51982,51982,51982,51982
Photo\tPhoto,14278,3342,14278,722,722,14278,14278,14278,14278,14278,...,14278,14278,14278,14278,14278,14278,14278,14278,14278,14278
Photo\tPhoto\tPhoto,3906,1180,3906,338,338,3906,3906,3906,3906,3906,...,3906,3906,3906,3906,3906,3906,3906,3906,3906,3906
Photo\tPhoto\tPhoto\tPhoto,7419,2354,7419,537,537,7419,7419,7419,7419,7419,...,7419,7419,7419,7419,7419,7419,7419,7419,7419,7419
Photo\tVideo,2,1,2,0,0,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Video,17976,5310,17976,1827,1827,17976,17976,17976,17976,17976,...,17976,17976,17976,17976,17976,17976,17976,17976,17976,17976
Video\tVideo,4,2,4,1,1,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [6]:
from preprocess import preprocess, get_dask_compute_environment, load_all_preprocessed_data
from dask.distributed import progress

In [2]:
_, delayed_save = preprocess()

The following preprocessed columns are dumped:  ['has_reply', 'bert_token_len', 'has_retweet', 'has_retweet_comment', 'has_like', 'n_photos', 'n_videos', 'n_gifs', 'reply_age', 'TE_language_has_reply']
Feature generation ready, associated delayed objects: Dask DataFrame Structure:
               bert_base_multilingual_cased_tokens hashtags tweet_id  medias   links domains    type language timestamp a_user_id a_follower_count a_following_count a_is_verified a_account_creation b_user_id b_follower_count b_following_count b_is_verified b_account_creation a_follows_b   reply retweet retweet_comment    like has_reply bert_token_len has_retweet has_retweet_comment has_like n_photos n_videos  n_gifs reply_age TE_language_has_reply
npartitions=96                                                                                                                                                                                                                                                            

In [3]:
with get_dask_compute_environment() as client:
    f = client.persist(delayed_save)
    progress(f)
    f.compute()

Compute environment established: <Client: 'tcp://127.0.0.1:37097' processes=3 threads=6, memory=11.18 GiB>


KeyboardInterrupt: 

In [1]:
from preprocess import get_dask_compute_environment, load_all_preprocessed_data, conditional_probabilities, load_default_config
from dask.distributed import progress
df= load_all_preprocessed_data()
df

,bert_base_multilingual_cased_tokens,hashtags,tweet_id,medias,links,domains,type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like,has_reply,bert_token_len,has_retweet,has_retweet_comment,has_like,n_photos,n_videos,n_gifs,reply_age,TE_language_has_reply
npartitions=96,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,object,object,object,object,object,object,object,object,uint32,object,uint32,uint32,bool,uint32,object,uint32,uint32,bool,uint32,bool,uint32,uint32,uint32,uint32,bool,uint32,bool,bool,bool,float64,float64,float64,uint32,float64
258290,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23921808,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24087220,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [2]:
cfg = load_default_config()
f = conditional_probabilities(df, cfg['marginal_prob_columns']['features'], cfg['marginal_prob_columns']['per_labels'])

In [3]:
client = get_dask_compute_environment(cfg)
f = client.persist(f)
f.compute()

Compute environment established: <Client: 'tcp://127.0.0.1:43367' processes=2 threads=4, memory=13.04 GiB>


has_reply  has_retweet  has_retweet_comment  has_like  a_follows_b  b_is_verified  a_is_verified  type      language                        
False      False        False                False     False        False          False          Quote     00304D7356D6C64481190D708D8F739C    5.215897e-05
                                                                                                            105008E45831ADE8AF1DB888319F422A    4.449580e-06
                                                                                                            10C6C994C2AD434F9D49D4BE9CFBC613    2.331909e-05
                                                                                                            159541FA269CA8A9CDB93658CAEC4CA2    2.101191e-05
                                                                                                            1F73BB863A39DB62B4A55B7E558DB1E8    1.221245e-03
                                                                          

In [4]:
a = f.compute()

In [9]:
a.to_csv("./conditional_probs.csv")

In [27]:
import pandas as pd
df = pd.read_parquet('preprocessed/part.0.parquet')
df2 = pd.read_parquet('preprocessed_features/part.0.parquet')
df3 = df.merge(df2, how='inner', left_index=True, right_index=True)

In [25]:
df3.index.name

'idx'

In [47]:
cfg = load_default_config()
labels = cfg['marginal_prob_columns']['per_labels']
marg = df3[labels+["tweet_id"]].groupby(labels)["tweet_id"].count().rename('marginal').to_frame()
marg

marginal
has_reply has_retweet has_retweet_comment has_like          
False     False       False               False       130482
                                          True         97185
                      True                False         1019
                                          True           175
          True        False               False        17694
                                          True          3924
                      True                False          524
                                          True           128
True      False       False               False         6022
                                          True           955
                      True                False            8
          True        False               False          100
                                          True            61
                      True                False            9
                                          True             3

In [48]:
features = cfg['marginal_prob_columns']['features']

In [49]:
cond = df3[labels+features+['tweet_id']].groupby(labels+features)['tweet_id'].count().rename('conditional').to_frame()
cond

conditional
has_reply has_retweet has_retweet_comment has_like a_follows_b b_is_verified a_is_verified type     language                                     
False     False       False               False    False       False         False         Quote    00304D7356D6C64481190D708D8F739C            9
                                                                                                    105008E45831ADE8AF1DB888319F422A            1
                                                                                                    10C6C994C2AD434F9D49D4BE9CFBC613            6
                                                                                                    159541FA269CA8A9CDB93658CAEC4CA2            5
                                                                                                    1F73BB863A39DB62B4A55B7E558DB1E8          152
...                                                                                                                                           ...
True      True        True                False    True        False         False         TopLevel B0FA488F2911701DD8EC5B1EA5E322D8            1
                                                                                                    B8B04128918BBF54E2E178BFF1ABA833            1
                                          True     False       False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1
                                                                             True          TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1
                                                   True        False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1

[2366 rows x 1 columns]

In [51]:
p = cond.join(marg, on=labels, how='left')
p

conditional  \
has_reply has_retweet has_retweet_comment has_like a_follows_b b_is_verified a_is_verified type     language                                        
False     False       False               False    False       False         False         Quote    00304D7356D6C64481190D708D8F739C            9   
                                                                                                    105008E45831ADE8AF1DB888319F422A            1   
                                                                                                    10C6C994C2AD434F9D49D4BE9CFBC613            6   
                                                                                                    159541FA269CA8A9CDB93658CAEC4CA2            5   
                                                                                                    1F73BB863A39DB62B4A55B7E558DB1E8          152   
...                                                                                                                                           ...   
True      True        True                False    True        False         False         TopLevel B0FA488F2911701DD8EC5B1EA5E322D8            1   
                                                                                                    B8B04128918BBF54E2E178BFF1ABA833            1   
                                          True     False       False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1   
                                                                             True          TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1   
                                                   True        False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA            1   

                                                                                                                                      marginal  
has_reply has_retweet has_retweet_comment has_like a_follows_b b_is_verified a_is_verified type     language                                    
False     False       False               False    False       False         False         Quote    00304D7356D6C64481190D708D8F739C    130482  
                                                                                                    105008E45831ADE8AF1DB888319F422A    130482  
                                                                                                    10C6C994C2AD434F9D49D4BE9CFBC613    130482  
                                                                                                    159541FA269CA8A9CDB93658CAEC4CA2    130482  
                                                                                                    1F73BB863A39DB62B4A55B7E558DB1E8    130482  
...                                                                                                                                        ...  
True      True        True                False    True        False         False         TopLevel B0FA488F2911701DD8EC5B1EA5E322D8         9  
                                                                                                    B8B04128918BBF54E2E178BFF1ABA833         9  
                                          True     False       False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA         3  
                                                                             True          TopLevel 488B32D24BD4BB44172EB981C1BCA6FA         3  
                                                   True        False         False         TopLevel 488B32D24BD4BB44172EB981C1BCA6FA         3  

[2366 rows x 2 columns]

In [54]:
pp = p.conditional / p.marginal
pp

pandas.core.series.Series

In [53]:
pp.sum()

15.0